In [3]:
import numpy as np
import matplotlib.pyplot as plt
from queue import Queue
from numpy.random import exponential as rexp
import pandas as pd

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [219]:

    
    
class Order_Grid:
    def __init__(self, smallest_price, largest_price, unit = 1):
        self._price_grid = list(range(smallest_price, largest_price+unit, unit))[::-1] # should consider use np.arange but need to deal with data type.
        self.unit = unit
        price_grid = self._price_grid
        self.min_price = min(price_grid)
        self.max_price = max(price_grid)
        self.n = len(price_grid)
        self.X = pd.Series([0 for _ in price_grid], index=price_grid)
        self.first_bid = min(price_grid)
        self.first_ask = max(price_grid)
        self.update_papb()
        self.bsize = 0
        self.asize = 0
        
    def add_market(self, qty, type_, limit_price = None):
        q = qty
        i = 0
        unit = self.unit
        if type_=='buy':
            assert qty <= self.bsize, f'no enough liquidity: {self.bsize}'
            isbuy, sign, best_price = True, 1, self.pA
        elif type_ == 'sell':
            assert qty <= self.asize, f'no enough liquidity: {self.asize}'
            isbuy, sign, best_price = False, -1, self.pB
        else:
            raise TypeError(f"The second parameter type must be buy or sell, invalid input: {type_}")
        while q > 0:
            idx = best_price+sign*i*unit
            exec_q_at_i = min(sign*self.X[idx], q)
            self.X[idx] -= sign*exec_q_at_i
            if isbuy: self.asize -= exec_q_at_i
            else: self.bsize -= exec_q_at_i
            q = q-exec_q_at_i
            if limit_price:
                if idx == limit_price and q > 0:
                    assert self.X[idx] == 0, f'{self.X[idx]}'
                    self.X[idx] = -q
                    break
            i += 1
        self.update_papb()
        
    def cancel(self, price, qty):
        if price<= self.pA:
            cancelled_order = max(min(qty, -self.X[price]), 0)
            self.X[price] += cancelled_order
            self.bsize -= cancelled_order
        elif price >= self.pB:
            cancelled_order = max(min(qty, self.X[price]), 0)
            self.X[price] -= cancelled_order
            self.asize -= cancelled_order
        else:
            raise
        
        self.update_papb()
            
            
    def add_limit(self, price, qty, type_):
        assert qty>=0
        if type_=='buy':
            isbuy, sign, best_price = True, 1, self.pA
            if price >= self.pA:
                self.add_market(qty, type_, limit_price = price)
            else:
                self.X[price] -= sign*qty
                self.bsize += qty
        elif type_ == 'sell':
            isbuy, sign, best_price = False, -1, self.pB
            if price <= self.pB:
                self.add_market(qty, type_, limit_price = price)
            else:
                self.X[price] -= sign*qty
                self.asize += qty
        else:
            raise TypeError(f"The third parameter type must be buy or sell, invalid input: {type_}")           
        
        self.update_papb()

    def update_papb(self):
        pa = self.X[self.X>0].index.min()
        pb = self.X[self.X<0].index.max()
        self.pA = self.max_price if pd.isna(pa) else pa
        self.pB = self.min_price if pd.isna(pb) else pb
        assert self.pA >= self.pB, f'pA: {self.pA}; pB: {self.pB}'
        self.pM = (self.pA+self.pB)*0.5
        self.spread = self.pA -self.pB
        
    
    def __repr__(self):
        return repr(self.X)
    

In [26]:
pd.isna(pd.Series([1]).index.min())

False

In [220]:

g = Order_Grid(1, 10)

In [221]:
g.bsize, g.asize

(0, 0)

In [222]:
g.pA, g.pB

(10, 1)

In [227]:
g.X

10      0
9       0
8     150
7     150
6      50
5     -50
4    -200
3    -200
2       0
1       0
dtype: int64

In [224]:
g.add_limit(8, 150, 'sell')
g.add_limit(7, 150, 'sell')
g.add_limit(6, 50, 'sell')
g.add_limit(5, 100, 'sell')
g.add_limit(4, 200, 'buy')
g.add_limit(3, 100, 'buy')
g.add_limit(3, 100, 'buy')

In [207]:
g.cancel(3,300)

In [226]:
g.add_limit(5, 150, 'buy')

In [ ]:
g

In [214]:
g.add_market(350, 'buy')

In [69]:
g.X

10      0
9       0
8     150
7      50
6     100
5    -100
4    -200
3     -50
2     -50
1       0
dtype: int64

In [63]:
g.spread

1

In [4]:

            
            
class Dark_Pool:
    def __init__(self, bid_rate, ask_rate, total_time, matching_chance):
        self.bid_rate = bid_rate
        self.ask_rate = ask_rate
        self.total_time = total_time
        self.bid_qsize = 0
        self.ask_qsize = 0
        self.matching_chance = matching_chance
        self.next_bid = rexp(self.bid_rate)
        self.next_ask = rexp(self.ask_rate)
        self.current_tick = 0
        self.order_queue = Order_List()

    
    def _count_orders(self, queue_size, next_time, rate, price, side = None):
        c = 0
        next_time -= 1
        if next_time < 0:
            while True:
                c += 1
                queue_size += 1
                self.order_queue.add(Order(price, side, 100, self.current_tick), side)
                next_time += rexp(rate)
                if next_time >= 0:
                    break
        else:
            pass
        
#         print(f'{side} new order: {c}')
        return queue_size, next_time
    
    def matching(self, price):
        delta = self.bid_qsize - self.ask_qsize
        ret = min(self.bid_qsize, self.ask_qsize)
        if delta <= 0:
            self.bid_qsize, self.ask_qsize = (0, -delta)  
        else:
            self.bid_qsize, self.ask_qsize = (delta, 0)
            
#         print(f'num of filled order: {ret}, left order: {delta}')
        self.order_queue.mark_fill(ret, price, self.current_tick)
        return ret
        
    def count_bid_ask(self, price):  
        self.bid_qsize, self.next_bid = self._count_orders(self.bid_qsize, self.next_bid, 
                                                           self.bid_rate, price, 'buy')
        
        self.ask_qsize, self.next_ask = self._count_orders(self.ask_qsize, self.next_ask, 
                                                           self.ask_rate, price, 'sell')
            

    def main(self, mid_price_seq):
        if len(mid_price_seq)<self.total_time: raise
        
        exec_size_ls = [0]
        nb = [0]
        na = [0]

        for t in range(self.total_time)[1::]:
            self.current_tick = t
#             print(self.current_tick)
            self.count_bid_ask(mid_price_seq[t])
            exec_size_ls.append(self.matching(mid_price_seq[t]) if np.random.uniform() <= self.matching_chance else 0)
            nb.append(self.bid_qsize)
            na.append(self.ask_qsize)
        return np.array(exec_size_ls)*100, np.array(nb)*100, np.array(na)*100, {}